In [12]:
%pwd

'/Users/myrtle/skewl/cs 429 info retrieval/project/indexer'

In [13]:
%cd ..

/Users/myrtle/skewl/cs 429 info retrieval/project


In [19]:
from pprint import pprint

In [81]:
import re
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [88]:
np.finfo(np.longdouble)

finfo(resolution=1.0000000000000000715e-18, min=-1.189731495357231765e+4932, max=1.189731495357231765e+4932, dtype=float128)

In [4]:
def temp_tokenizer(text: str) -> list[str]:
    toks = re.split(r"\s", text)
    toks = [s for s in toks if len(s) > 0]
    return toks

In [5]:
def temp_preprocessor(text: str) -> str:
    text = text.strip()
    text = re.sub(r'[^a-zA-Z0-9_\-\s]', '', text)
    text = "".join(ch for ch in text if ord(ch) <= 127)
    return text

In [5]:
t = "holaˈl̪ˠ"
"".join(ch for ch in t if ord(ch) <= 127)

'holal'

In [ ]:
"""from 20newsgroups data"""

from sklearn.datasets import fetch_20newsgroups


news = fetch_20newsgroups(subset="all", shuffle=False, remove=("headers", "footers", "quotes"))


# documents = ["my", "list", "of", "docs"]
documents = news.data


#####   STOLEN FROM https://stackoverflow.com/questions/68003003/python-sklearn-tfidfvectorizer-vectorize-documents-ahead-of-query-for-semantic

vectorizer = TfidfVectorizer(preprocessor=temp_preprocessor, tokenizer=temp_tokenizer)


# fit_transform does two things: fits the vectorizer and transforms documents
doc_vectors = vectorizer.fit_transform(documents)





    
# inverted_doc_vectors = doc_vectors.transpose()

In [23]:
"""Okay let's just get the data from the csv file"""

import json

In [96]:
import linecache

line = linecache.getline("corpus/items.jsonl", 1)
doc = json.loads(line)
print(doc["url"])

https://en.wikipedia.org/wiki/List_of_common_misconceptions


In [104]:

documents = []

with open("corpus/items.jsonl", "r") as f:
    for doc_str in f.readlines():
        doc = json.loads(doc_str)
        documents.append(doc["text"])

print([d[:25] + "..." for d in documents[:10]])

#####   STOLEN FROM https://stackoverflow.com/questions/68003003/python-sklearn-tfidfvectorizer-vectorize-documents-ahead-of-query-for-semantic

vectorizer = TfidfVectorizer(
    preprocessor=temp_preprocessor, 
    tokenizer=temp_tokenizer
)

doc_vectors = vectorizer.fit_transform(documents)

pprint(doc_vectors)


['Each entry on this list o...', 'The Archbasilica of Saint...', 'In lexical semantics, opp...', 'A DOI aims to resolve to ...', 'The influence of the art ...', 'The International Standar...', 'Water vapor, water vapour...', 'Hunter Stockton Thompson ...', 'In 1994, The Chronicle of...', 'From the invention of mov...']
<100x34006 sparse matrix of type '<class 'numpy.float64'>'
	with 89847 stored elements in Compressed Sparse Row format>


In [99]:

search_terms = "Jesus"
k = 5

search_term_vector = vectorizer.transform([search_terms])

# REPLACE THIS ! ! ! ! !

cosine_similarities = linear_kernel(doc_vectors, search_term_vector).flatten()

doc_inds_by_score_reversed = np.argsort(cosine_similarities)
top_k_inds_by_score_reversed = doc_inds_by_score_reversed[len(doc_inds_by_score_reversed)-k:]
top_k_inds_by_score = top_k_inds_by_score_reversed[::-1]

print(top_k_inds_by_score)


[34 49  1 26 77]


In [94]:

def print_doc(index, score, text):
    
    doc_words = re.split(r"\s", text)
    
    # line_len = 74
    line_len = 10
    def line_start(i):
        return line_len*i
    def line_end(doc, i):
        return min(line_start(i) + line_len, len(doc))

    # print(
    #     *(best_doc[line_start(i) : line_end(text, i)] 
    #       + ("" if (text[line_end(text, i)] == " " or text[line_end(text, i) - 1] == " ") else "-")
    #     for i in range(len(text) // line_len)),
    #     sep="\n"
    # )

    print(
        f"Best match (idx={index}, {score}):\n"
    )
    print(
        *(" ".join(doc_words[line_start(i) : line_end(doc_words, i)])
        for i in range(min(5, len(doc_words) // line_len))),
        sep="\n",
        end=""
    )
    if (len(doc_words) // line_len >= 5):
        print("...")

In [100]:
for i in top_k_inds_by_score:
    print_doc(i, cosine_similarities[i], documents[i])
    print()

Best match (idx=34, 0.02188715971168302):

The predecessor of the present church was probably built in
the early fourth century and that church was itself the
successor to one of the tituli, early Christian basilicas ascribed
to a patron and perhaps literally inscribed with his name.
Although nothing remains to establish with certainty where any of...

Best match (idx=49, 0.01751561775796079):

The earliest use of the term was in reference to
the canonical hour, also called the vigil, which was originally
celebrated by monks from about two hours after midnight to,
at latest, the dawn, the time for the canonical hour
of lauds (a practice still followed in certain orders). It...

Best match (idx=1, 0.007652297580682315):

The Archbasilica of Saint John Lateran (formally named the "Major
Papal, Patriarchal and Roman Archbasilica Cathedral of the Most Holy
Savior and Saints John the Baptist and the Evangelist in
Lateran, Mother and Head of All Churches in Rome and
in the World", and common